In [ ]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import mixture
from sklearn.metrics import log_loss

In [ ]:

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid


**Load the CIFAR-10 dataset**

In [ ]:
transform = transforms.ToTensor()

train_data = datasets.CIFAR10(root='../Data', train=True, download=True, transform= transform)
test_data = datasets.CIFAR10(root='../Data', train=False, download=True, transform= transform)


In [ ]:
train_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../Data
    Split: Train
    StandardTransform
Transform: ToTensor()

**Create loaders**

In [ ]:

torch.manual_seed(101)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

**Key Generation**



In [ ]:
S=6  # Number of classes ( or Gaussians)
s=2  # Number of selected Gaussians
N=6  # Length of our key 
GMM_classes=np.arange(S)  #Set GMM classes from 0 to 5
n_sampls=500
M=20*20 # number of features 
#model gmm
gmm = mixture.GaussianMixture(n_components=S)

#Generate random sampls
X=np.random.randn(n_sampls,M)
#fit the model
gmm.fit(X)

#Select s random indices from 1 to S 
def Select_Gaussian_Classes(GMM_classes,s):
  GMM_classes=np.arange(S) 
  T=np.random.choice(GMM_classes,size=s,replace=False)
  return T


def subset_training_Data(T,X_train,Y_train):
  Y_pred=gmm.predict(X_train)
  indice=np.vstack([np.argwhere(Y_pred==t) for t in T]).flatten()
  nsampls=int(len(X_train)/100)
  key_index=np.random.choice(indice,size=nsampls,replace=False)
  X_key,Y_key=X_train[key_index], Y_train[key_index]
  return key_index,X_key,Y_key


def generate_secrect_matrix(M,N):
  return np.random.randn(M,N)

def sigmoid(x):
  return 1/(1+np.exp(-x))

def Hard_Th(G):
  return (G>0.5 )*1


Y=gmm.predict(X)
T=Select_Gaussian_Classes(GMM_classes,s)
key_index,X_key,Y_key=subset_training_Data(T,X,Y)


In [ ]:
#Key generation algorithm
A=generate_secrect_matrix(M,N)
mu=gmm.means_         # means of selected Gaussians
G=sigmoid(np.dot(mu,A))
b_model=Hard_Th(G)
b_model       #The key generated

array([[1, 1, 0, 0, 0, 1],
       [1, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [0, 1, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 1]])

**Fine-Tuning**

In [ ]:
# Quadratic error
def mse(X1,X2):
  return ((X1-X2)**2).mean()



#Computing loss1 

def loss1(activation_map,mu,mu_T):
 T=Select_Gaussian_Classes(GMM_classes,s)
 T_=[i for i in range(S) if i not in T]
 loss_=np.array([mse(mu[i],mu[j]) for i in T for j in T_]).mean()
 return mse(activation_map,mu_T)-loss_


In [ ]:
mu_T=mu[Y][key_index]          # means of X_key
activation=np.random.randn(mu_T.shape[0],mu_T.shape[1]) # Generate random activation 
loss1(activation,mu)                                    #Testing loss1

0.9168328304129281

In [ ]:
# Computing loss2

def loss2(b,G):
  return log_loss(b,G)

In [ ]:
loss2(b_model,G)

3.8020090565821083